<a href="https://colab.research.google.com/github/Vishwa1030/Text-summarization/blob/main/Copy_of_Summarization_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <b>Summarize medicine texts using Machine Learning</b>

## 1. Download and import libraries

In [ ]:
!pip install transformers==2.2.2#for abstractive
!pip install bert-extractive-summarizer #for extractive

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement transformers==2.2.2#for (from versions: 0.1, 2.0.0, 2.1.0, 2.1.1, 2.2.0, 2.2.1, 2.2.2, 2.3.0, 2.4.0, 2.4.1, 2.5.0, 2.5.1, 2.6.0, 2.7.0, 2.8.0, 2.9.0, 2.9.1, 2.10.0, 2.11.0, 3.0.0, 3.0.1, 3.0.2, 3.1.0, 3.2.0, 3.3.0, 3.3.1, 3.4.0, 3.5.0, 3.5.1, 4.0.0rc1, 4.0.0, 4.0.1, 4.1.0, 4.1.1, 4.2.0, 4.2.1, 4.2.2, 4.3.0rc1, 4.3.0, 4.3.1, 4.3.2, 4.3.3, 4.4.0, 4.4.1, 4.4.2, 4.5.0, 4.5.1, 4.6.0, 4.6.1, 4.7.0, 4.8.0, 4.8.1, 4.8.2, 4.9.0, 4.9.1, 4.9.2, 4.10.0, 4.10.1, 4.10.2, 4.10.3, 4.11.0, 4.11.1, 4.11.2, 4.11.3, 4.12.0, 4.12.1, 4.12.2, 4.12.3, 4.12.4, 4.12.5, 4.13.0, 4.14.0, 4.14.1, 4.15.0, 4.16.0, 4.16.1, 4.16.2, 4.17.0, 4.18.0, 4.19.0, 4.19.1, 4.19.2, 4.19.3, 4.19.4, 4.20.0, 4.20.1, 4.21.0, 4.21.1, 4.21.2, 4.21.3, 4.22.0, 4.22.1, 4.22.2, 4.23.0, 4.23.1, 4.24.0)
ERROR: No matching distribution found for transformers==2.2.2#for
Looking in ind

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from transformers import pipeline
from summarizer import Summarizer
from nltk import sent_tokenize
import re

## 2. Function for reading text from file

In [ ]:
def read_text(path_to_file):
  f = open(path_to_file, 'r', encoding='utf8')
  text = f.read()
  f.close()
  return text

## 3. Text preprocessing function

In [ ]:
def preprocess_text(text):
    text = text.strip()
    text = text.replace('\n', ' ').replace('\r', '')
    text = re.sub('\.+', '.', text)
    text = text.replace('«', '"').replace('»', '"')
    text = re.sub('[\.\?\!\;]"', '"', text)
    text = re.sub('[\.\?\!]\;"', ';', text)
    text = re.sub(' +', ' ', text)
    text = re.sub('\t+', ' ', text)
    return text


def smart_tokenization(text):
    sentences = sent_tokenize(text)
    res = []
    cur_text = ''
    for sent in sentences:
        #2000 - max count of symbols for abstractive model
        if len(cur_text + sent) > 2000:
            res.append(cur_text)
            cur_text = sent
        else:
            cur_text += sent
    if cur_text != '':
      res.append(cur_text)
    return res

## 4. Define classes of summarizers

In [ ]:
class AbstractiveSummarizer():
    def __init__(self):
        self.model = pipeline('summarization')

    def generate_summary(self, text):
        text = preprocess_text(text)
        parts = smart_tokenization(text)
        res = []
        for part in parts:
            res.append(self.model(part)[0]['summary_text'])
        return ' '.join(res)


class ExtractiveSummarizer():
    def __init__(self):
        self.model = Summarizer()

    def generate_summary(self, text):
        text = preprocess_text(text)
        return self.model(text)

## 5. Now we can summarize texts!

In [ ]:
TEXT_TO_SUMMARIZE = """
Introduction
The etiology of alopecia areata (AA), a chronic inflammatory
disease that affects the hair follicle and sometimes the nails, has not
yet been fully understood. Besides the genetic background, nonspecific immune response, organ-specific autoimmune reactions,
and environmental factors are among the most frequently discussed
issues in the pathogenesis of this disease [1].
It has long been claimed that AA is a psychosomatic disease
triggered by stressful life events [2]. In a study investigating
comorbid diseases in patients with AA, prevalence of depression
and anxiety was found to be high (25.5%) in an 11-year period [3].
On the other hand, depression may be present before AA [4].
Family problems, work-related problems, and mourning have
also been reported by AA patients [4]. It has been suggested that
neuroendocrine immunology as well as psychosocial factors may
be responsible for the association between AA and depression [5].
It is well known that sleep has important effects on immunity.
There is evidence of a relationship between sleep disorders and
autoimmune diseases including rheumatoid arthritis, ankylosing
spondylitis, Sjögren’s syndrome and systemic lupus erythematosus
[6]. It has been reported that patients with sleep disorders, especially
young people, had an increased risk of AA [7]. In the same study,
sleep disorders were also associated with other autoimmunity
related diseases such as Graves’ disease, Hashimoto’s thyroiditis,
vitiligo and rheumatoid arthritis.
Alopecia areata may occur in different clinical types. In the most
common type, there is hair loss in the scalp[8]. If all of the scalp
hair is involved, it is called alopecia totalis and, if all of the body
hair is involved, it is called alopecia universalis. In the ophiasis
type hair loss is seen at the junction of occipital hair line and dermis, and in the reticular type the hair loss is reticular [8].
Nail changes are seen in 30% of patients with AA [9]. The most
common nail finding is small pits, and the other findings are
the Beau line, longitudinal line, koilonychia, onychorhexis,
onychomadesis, leukonychia, red-staine lunula, and other nail
defects can [9]. It has been suggested that nail change is related to
the severity of the disease and indicates possible resistance [10].
It is hard to predict natural course in alopecia areata. In some
patients, hair completely reappears, in some others, some
improvement in AA occurs, and, in still others, the hair loss further
increases [11]. Poor prognostic criteria include presence of atopia,
extensive involvement, family history, onychodystrophy, presence
of disease for more than 5 years, autoimmune disease, ophiasis
type, asthma, and the presence of nevus flammeus, allergic rhinitis,
and atopic dermatitis [12].
The aim of our study was to evaluate the frequency of depression
and sleep quality in patients with AA, and to investigate the
relationship between the clinical characteristics of the disease such
as duration, severity, and type of illness and depression, and sleep
quality.
Material and Methods
Sample
This study was conducted in Skin and Venereal Diseases
Department of the Medical School of Hitit University. This study
included 52 patients above 18 years of age who admitted to the
dermatology clinic and were diagnosed as alopecia areata and 51
healthy volunteers. Healthy volunteers were selected from hospital
staff and the relatives of hospital staff who didn’t have any history
of psychiatric, systemic or dermatological disease and agreed to
participate in the study. The patients and control group included
in the study were informed about the study, and the questionnaires
were completed after obtaining their written informed consents.
Illiterate patients and those couldn’t fill the scales due to cognitive
dysfunction were not included in the study. Patients who were
being treated for a systemic disease or who had treatment for a
psychiatric disease during the last year were excluded. This study
was approved by the local ethics committee and performed in
accordance with the ethical standards of the Helsinki Declaration.
Instruments
Beck Depression Inventory (BDI): This scale has been developed
by Beck et al and aims to measure the severity of depression[13].
The scale consists of 21 items and is scored between 0 to 3.
Pittsburgh Sleep Quality Index (PSQI): It was developed by Buysse
et al.[14] and consists of 24 questions, 19 questions are filled by
the subject himself and 5 questions are filled by the spouse or a
room partner. From the scored questions of the scale 7 subscale
scores are obtained: subjective sleep quality, sleep latency, sleep
duration, habitual sleep efficiency, sleep disturbance, use of sleep
medications, and daytime dysfunction. The total score of seven
subscales gives total scale score which varies between 0 and 21.
Severity of Alopecia Areata Tool; SALT: This scale was developed
by Olsen et al. [15] and calculated by measuring how much of the
total body surface is affected considering involved body parts. In
this study, patients with hair loss below 25% were classified as
mild, 25-50% moderate and above 50% as severe loss.
Procedures
All of the included cases were informed about the study, informed
consent forms were obtained, and then sociodemographic data
form, BDI, and PSQI were filled by the patients. In addition to
these, information about the severity of alopecia areata, duration
of the disease, treatments they received, and information about
their illnesses were also recorded.
Statistical Analysis
To summarize data obtained from the study, descriptive statistics
were given as mean ± standard deviation or median and
interquartile ranges depending on the distribution for continuous
variables. The normality of the numerical variables was checked
by the Kolmogorov-Smirnov test. Independent Samples t test
was used in the comparison of two independent groups when
the numerical variables showed normal distribution, and Mann
Whitney U test was used otherwise. In comparison of more than
three independent groups, one-way ANOVA was used where the
numerical variables were normally distributed and Kruskall Wallis
test was used otherwise. For between group differences, Tukey’s
test was used when the distribution of the data was homogenous
and Games-Howell test was used otherwise. Between group
differences in nonparametric tests were evaluated with DwassSteel-Critchlow-Fligner test. Pearson Chi-Square test was used for
categorical variables, and Fisher’s exact test was used for RXC
tables. To evaluate the associations between numerical variables,
Pearson test was used when the data were normally distributed
and Spearman’s Rho was used otherwise. Statistical analyses were
performed with Jamovi Project software (2019; Version 0.9.5.12)
and p<0.05 was considered to be significant in statistical analyses.
Results
This study included 52 AA patients and 51 controls. There was
no significant difference in age, sex, education status, cigarette
smoking, height, weight, and body mass index (BMI) values
between the groups (Table 1).
Alopecia areata was localized only to scalp hair in 24 patients
(%46.2). In 20 (38.5%) patients the disease duration was less than
1 month. In 29 patients (55.8%) single, patchy AA was found.
According to SALT, the severity of AA was mild in 46 (86.4%)
of the patients. The number of patients who had at least one poor
prognostic criteria was 28 (53.8%). The most common poor
prognostic criterion was family history (n=10, 19.2%). Beginning
age of the disease was mean 27.5 (± 7.8) (Table 2).
Median BDI total score of AA patients (8.5) was higher than that of
the control group (2) (p<0.001). In AA patients, besides PSQI total
score, subjective sleep quality, sleep latency, sleep disturbance,
and daytime dysfunction subscale scores were statistically higher
than those of the control group (p<0.001, p=0.002, p=0.001, and
p<0.001 respectively).
Table 1. Comparisons of age, sex, education status, cigarette smoking, height, weight, and BMI values between the groups
Group
Patient (n=52) Control (n=51) p
Age, Mean ± SD 28.8 ± 7.7 29.3 ± 8.7 0.792
Sex Male. n (%) 40 (76.9) 40 (78.4)
0.854
Female. n (%) 12 (23.1) 11 (21.6)
Education status Primary school. n (%) 6 (11.5) 4 (7.8)
0,637
Secondary school. n (%) 6 (11.5) 5 (9.8)
High school. n (%) 19 (36.5) 15 (29.4)
University. n (%) 21 (40.4) 27 (52.9)
Cigarette smoking No. n (%) 27 (51.9) 31 (60.8)
0.365
Yes. n (%) 25 (48.1) 20 (39.2)
Height (cm), Mean± SD 174.3 ± 9.3 171.4 ± 8.6 0.107
Weight (kg), Mean ± SD 70.5 ± 11.6 71.3 ± 11.6 0.741
BMI (kg/m2), Mean ± SD 23.2 ± 3.3 24.1 ± 2.6 0.103
BMI: body mass index, SD: standard deviation
Table 2. Clinical features of the individuals
n (%)
Localization of alopecia areata Only scalp 24 (46.2)
Only beard 18 (34.6)
Only scalp, eyelashes 5 (9.6)
Scalp+ beard 4 (7.7)
Scalp + eyelashes + eyebrows 1 (1.9)
Type of alopecia areata Single patch 29 (55.8)
Multiple patches 23 (44.2)
Alopecia areata severity (Mild) Hair loss < %25 46 (88.5)
(Moderate) Hair loss %26-50 5 (9.6)
(Severe) Hair loss %50 1 (1.9)
Onychodistrophy No 45 (86.54)
Yes 7 (13.46)
Poor prognostic criteria No 28 (53.8)
Yes 24 (46.2)
Atopy history 8 (15.4)
Family history 10 (19.2)
Longer than 5 years 3 (5.8)
Asthma 3 (5.8)
Allergic rhinitis 4 (7.7)
Atopic dermatitis 3 (5.8)
Age of onset of disease 27.5 ± 7.8
Previous treatments No 36 (69.2)
Topical 8 (15.4)
Intralesion 7 (13.5)
Systemic 1 (1.9)
Onychodystrophy was detected in 7 patients (13.46%). As
onychodystrophy type, 5 patients (9.6%) had pitting, 1 patient
had longitudinal striation, and 1 patient (1.9%) had opaque nail.
No difference could be found between patients who had or who
didn’t have onychodystrophy in BDI, PSQI total score, and PSQI
subscale scores.
Comparisons for Beck Depression Inventory scores demonstrated
that the localization of alopecia areata didn’t cause any effect
(p=0.303). BDI scores of the patients who had moderate or severe
alopecia areata were higher than those of the patients who had
mild disease (p=0.023). No difference could be found in BDI score
according to the presence or absence of poor prognostic criteria.
Evaluation for the presence of poor prognostic criteria separately
demonstrated that the BDI score was higher in the presence of
only atopic dermatitis (p=0.016). Prior treatment did not affect the
mean BDI score (Table 3).
Comparisons for the Pittsburgh Sleep Quality Index (PSQI)
revealed no difference for alopecia areata localization. Although
scores were higher in patients with moderate or severe alopecia
areata, the difference couldn’t reach statistical significance
(p=0.092). No difference could be found in PSQI scores according
to the presence or absence of poor prognostic criteria. Evaluation
of the PSQI scores in the presence of separate poor prognostic
criteria revealed that the presence of alopecia areata for more than
5 years (p=0.001) and presence of atopic dermatitis (p=0.016)
were associated with higher PSQI scores (Table 3).
Comparisons in terms of gender revealed that in the patient group
the median BDI (16) and PSQI (7.5) scores in females were
statistically significantly higher than BDI (7.5) and PSQI (6)
scores in males (p=0.026 and p=0.012, respectively). Also in the
control group the median BDI (9) and PSQI (4) scores in females
were statistically significantly higher than BDI (1.5) and PSQI (2)
scores in males (p=0.001 and p=0.035, respectively)
Table 3. Association between the clinical features of alopecia areata and BDI, PSQI scores
BDI p PSQI p
Localization of alopecia areata
Limited to scalp, (n=24) 9 [3.5 – 16]
0.303
7.5 [4.5 – 9]
0.740
Limited to beard, (n=18) 7 [4 – 16] 5.5 [5 – 8]
Only eyebrows, eyelashes, (n=5) 17 [13 – 19] 7 [6 – 13]
Scalp + beard, (n=4) 11 [9 – 15] 7 [3 – 11.5]
Scalp + eyebrows, eyelashes (n=1) α 1 [1 – 1] 5 [5 – 5]
Severity of alopecia areata
Mild hair loss 25%, (n=46) 8 [4 – 15]
0.023*
6 [4 – 8]
0.092
Moderate hair loss 26-50%, (n=6) 17.5 [12 – 18] 9 [9 – 9]
Poor prognostic criteria
No, (n=28) 8.5 [4.5 – 15.5]
0.576
6 [4.5 – 9]
0.372
Yes, (n=24) 9 [4 – 17.5] 7 [5 – 9]
Poor prognostic criteria
Atopy, (n=8) 12.5 [6.5 – 23] 0.361 8 [5.5 – 10.5] 0.417
Family history, (n=10) 6 [2 – 19] 0.862 8.5 [5 – 13] 0.160
Onychodystrophy (n=7) 17 [6 – 18] 0.243 7 [5 – 9] 0.639
More than 5 years (n=3) 6 [4 – 12] 0.569 5 [4 – 5] 0.001*
Asthma (n=3) 6 [0 – 15] 0.376 4 [2 – 7] 0.184
Allergic rhinitis (n=4) 8.5 [3.5 – 12.5] 0.570 7.5 [4.5 – 8] 0.671
Atopic dermatitis (n=3) 28 [18 – 39] 0.016* 12 [9 – 14] 0.016*
Previous treatments
No, (n=36) 8 [4 – 15]
0.393
6 [5 – 8.5]
0.434
Yes, (n=16) 11 [5 – 18.5] 7 [4.5 – 10]
*p<0.05 Mann Whitney U test was used. Descriptive statistics were given as median [IQR].
α: Not included in analyses. AA, alopecia areata; BDI: Beck Depression Inventory, IQR: Interquartile Range, PSQI: Pittsburgh Sleep Quality Index
Discussion
In our study sleep quality of alopecia areata patients which was
measured by PSQI scale was lower than that of the control group.
There may be a two-way relationship between sleep and AA. Sleep
deprivation was reported to increase serum interleukin (IL)-1 and
tumor necrosis factor in mice[16]. By this way, decreased sleep
quality might induce autoimmunity and cause AA in addition to
other autoimmune diseases. On the other hand, AA may increase
susceptibility to depression by causing loss of self-esteem due to
induction of a loss in self-esteem [3]. In a previous study which
evaluated sleep quality in AA patients, the Epworth Sleepiness
Scale was applied to 105 AA patients and the mean ESS score was
found to be 5.66±3.93 [17]. Although there was no control group
in this study, considering that no difference could be found from
a previous study on the normal Japanese population, the sleep
quality in AA patients was hypothesized to be normal. Absence
of a control group is a major limitation of this study. This might
have prevented finding a significant result. To the best of our
knowledge, no previous study had evaluated sleep quality in AA
patients with PSQI.
In our study, the BDI scores in the AA group was higher than the
control group. This result suggests the findings of the previous
studies in the literature. Sellami et al used the Hospital Anxiety and
Depression Scale and found higher depression and anxiety scores
in the AA group than the control group [18]. Another controlled
study using the Hamilton Depression Scale found that depression
scores of the AA patients were higher than the control group [19].
The prevalence of onyschodystrophy in AA patients was reported
to be between 7-66% in previous studies [9]. The rate we detected
(13.46%) was in accordance with previous studies. Pitting, which
was the most common nail change in our study, has also been
reported as the most frequent nail change in most of the previous
studies[10]. Nail changes were more commonly reported in severe
AA forms like AA totalis (AAT) and AA universalis (AAU) [20].
In our study, the absence of more severe AA types like AAU and
AAT may have caused low rate of onychodystrophy and absence
of a relationship between the presence of onychodystrophy and the
PSQI and the BDI scores.
In our study, depression level was significantly higher in patients with severe disease measured by SALT; the disturbance in sleep
quality was also higher in these patients but the difference could
not reach statistical significance. In most of the studies in the
literature, severity of the psychiatric symptoms like depression and
anxiety increased with increasing severity of the disease [18,19].
We couldn’t find a relationship between the localization of AA
and depression or sleep quality. Aghaei et al. reported higher rates
of depression in AA patients with face involvement [21] which
couldn’t be found in other two studies [19,22].
No association could be found in our study between the beginning
age of the study or disease duration and depression or sleep
quality similar to the findings of previous studies [19,22]. Firooz
et al. reported longer disease duration was associated with higher
hopelessness about the treatment [23].
In most of the studies that evaluated depression and anxiety levels,
women were found to have higher levels than men [18,19,22].
This finding was evaluated by some authors as that the aesthetic
stress of women was higher than that of men [19]. In our study,
the level of depression in female AA patients was higher and sleep
quality was more impaired than male patients. However, we think
that it would be wrong to interpret this finding as AA has more
psychological effects in women. Because, in this age group, the
level of depression is higher in females than that in males and sleep
quality is more impaired [24]. However, the results emphasize the
importance of screening for women with AA in terms of psychiatric
symptoms.
In our study, the evaluation of the effect of poor prognostic criteria
on AA patients in depression and sleep quality revealed that the
presence of atopic dermatitis was associated with an increase in
depression level and deterioration in sleep quality. Considering the
association between atopic dermatitis and psychiatric symptoms,
this finding is not a surprise. The evaluation of sleep quality in
100 patients with atopic dermatitis by PSQI revealed poor sleep
quality [25].
Conclusion
The results of our study support the long-standing relationship
between AA and psychiatric disorders and indicate that sleep
quality is impaired as well as the increase in depression level. In
particular, a more careful assessment of female AA patients and
patients with comorbid atopic dermatitis is important in order not
to miss comorbid psychiatric problems.
"""

In [ ]:
TEXT_TO_SUMMARIZE = """
Digital version environment as a tool philological studies research gradually becomes an important part humanitarian issues scientific knowledge, simultaneously causing resistance professional level communities, despite achievements formal schools and structuralists in literary studies and folklore studies. Submit literary and folklore text in the form the relationship between components offered more V. Shklovsky and V. Propp. About decomposition its on "such components, of which his then you can it would be fun to collect for some reasons General rules", they say modern ones scientists [Zholkovsky, 1996, p. 51]. Question about if necessary "segmentation" ("cutting") text on "primary elements, motives, functions, that are in the chain create a composition works", puts and B. F. Yegorov and adds, what is this method produces "based on rich traditional experience of analysis, with the involvement of quite a lot creative intuitions" [Yegorov, 2001, p. 26-27]. In modern science is designated and begins to be filled in research application space, where to help creative philologist's intuitions digital messages are coming in research methods, demanding their own understanding and workings criteria working with philological texts data in the context "go to digital being the text that transforms it its principles perceptions and research working with a language block and literary material" [Tchevtaev, 1, 2019, p. 29].

Forms existence texts depend on from accepted ones in the society technologies storage and reproduction information. To change memory as a carrier information in non-written forms cultures new technologies have arrived letters and seals in the form of a manuscript and books, computer programs technologies caused an appearance digital text appearance Internet networks the main form existence text made hypertext, technology which allows you to share documents with the help of hyperlinks. At the end of the 90s, the inventor World World Wide Web sir Tim Berners-Lee expressed an idea about the need creating a new one network-semantic (Semantic Web), since the network hypertext links not machine readable and the search is reduced to to search for individual users words or phrases, while Semantic The network must link standing ones for texts meaning and data, making it possible analyze them automatically, what will allow go from search for words "towards understanding causes and consequences and relationships between things, events, processes, data flows”[ Gronas, 2018, p. 248.]. semantic structure of the Text the network must become new a form of existence the text in which will be implemented features new technologies processing options information.

In context digital development what is happening transition process from digital ones texts to semantic. Specialists it is believed that everything in the future text messages publications will become semantic links, by joining forces to the semantic page total network cultural heritage humanity. Creation Semantic The network requires views information in machine-readable format in the form, i.e. in the form of ordered ones and universal recognized users data structures. Literary resources texts like objects that represent cultural value from points of view aesthetic features, literary studies, historical data meanings and knowledge, they must be included in Semantic page network [ibid., p. 249.], but for the transformation philological studies knowledge in such a situation the form is required spend effort and time, although existing ones philological studies practices ‑ comments, indexes, detection parallel quotes places, establishment of authorship and dates, and p. a. – this is already drafting practices semantic page networks, but not machine-readable.

Semantic features digital editions, which should provide safety digital cultural assets objects that are the most important the sphere of the new multidisciplinary approach scientific directions Digital Humanities humanitarian issues science). Semantic field an edition is presentation text in the form related data that Express a straight line, explicit and understandable information for a computer program processing options relationship entities. Plain digital the text does not give features such processing, and he must be marked up, i.e. its elements assigned information about their meaning in the form of organized reports placemarks in machine-readable format format, and universal ‑ for markup purposes the same people entities in the different texts should be used the same ones placemarks. It universal the markup links data different texts in semantic network, so the subject semantic edition is not separate text or meeting texts, and all the set written texts [there same, p. 250-253.]. This the approach requires organization and support long-term data access in the digital format independently technological changes compatibility SOFTWARE and services used resources, security data storage [Bonch-Osmolovskaya, 2016, p. 1606]. Semantic features publications should rely on tools automatic translating data from the format critical ones devices of the past in file formats standardized markup. Semantic field an edition is the publication where the text contains additional "machine-readable" layer-markup. consisting of specific tags, using which are marked not just words, and their values, relationships, contexts, thanks to what researchers can be exchanged textological system information, embed texts in the global version digital cultural network, use various funds automatic computer processing for research works on the ratio facts and identification links in the text. Using ones and the same principles markup to different texts, you can create "universal the " language” to use writers and traditions will be able to talk with each other" [Tolstoy Digital].

For preparation semantic analysis digital text-based an edition that includes data markup and metadata, generally recognized encoding method is a standard TEI [TEI] - collectively developed by, supported community and used for presentation texts in digital format format. Standard TEI (Guidelines TEI principles: P5) includes tools encoding options (schemes, source code code) and recommendations for the solution various tasks processing options sources (documentation), published articles as a software solution software with an open window source code. The standard allows you to conduct formal encoding key features "text" document properties" [Skorinkin, 2016, p. 90], such as parameters manuscripts, linguistic information information, output data, editorial and critical review the device, etc.

IN THE RUSSIAN FEDERATION they are developing successfully semantic search projects text editions based on standards TEI, for example: Manuscript [Manuscript Portal] - digital archive of medieval documents Russian manuscripts; folklore Museum Bashkir language archive state-owned enterprise University [Eni " Folklore Bashkir language archive state-owned enterprise University"] "all the time topped up full text version network multifunctional collection folklore books records collected in the Republic Башкортостан. Special role in development digital ones semantic features literary publications Russian texts culture plays Tolstoy Digital project [Digital Tolstoy Digital project], which represents by itself. 90-volume edition collected works L. N. Tolstoy. This project began integration the texts of the Russian literature in the global digital cultural landscape, with the one hand, on the other hand, in the project was developed modern standards digital Russian publications goals the most important technical and conceptual requirements: 1) documentation the project should reflect the structure text corpora, its current value status, description of the meta tag and text based annotations; 2) the data must have a specific and clearly described format, to be published (if possible) in open mode access, including number to upload; 3) functionality the search engine must match described case layout texts; 4) digital the publication must strive submit knowledge that improve the quality primary users documents [Bonch-Osmolovskaya, 2016, p. 1609].

Institute Philology, journalism and cross-cultural communications SFU jointly with a laboratory Philology Department of Humanities research Southern scientific center center of the Russian Academy of Sciences they are working working on a project “Chekhov Digital” - digital publishing texts A. P. Chekhov. Goal project creation multi-level (semantic) meeting markup works of art A. p. Chekhov with relying on standards digital publications Text Encoding Initiative, creating digital infrastructure saves and research texts. The main one source – academic Complete collection essays and letters of Anton Chekhov in 30 volumes (Pssip) [Chekhov, 1974-1983], which it was prepared and released employees Institute world literature them. A. M. Gorky Academy of Sciences The USSR in the years 1974-1983 The publication consists of it consists of two parts - essays (in 18 volumes) and letters (in 12 volumes). Each that includes results textual and research work also need Supplement multilevel layout allowing to do editorial notes machine-readable and more attuned for research with the help of not only traditional approaches, but and computer methods.

Tasks project: 1) markup texts in the In TEI format; 2) creating a new file full database named data entities, mentioned in texts or comments/notes; 3) placement texts in the following format TEI in the open access for downloads and automatic processing; 4) creation specialized a site with flexible settings features search results and tools visualizations.

Texts A. P. Chekhov submitted by on various platforms resources in the different digital formats format. Most full version – 30-volume edition PSS and big volume of editorial and critical reviews materials, submitted by on the Feb resource [ENI "CHEKHOV"], however, the format most materials it is not machine readable and allows you to implement standard only search by keywords words. Except A. P. texts Chekhov and editorial and critical reviews notes/comments the Pssip includes a selection of it emails sent via the prism of which social networks writer's connections they may be studied by computer programs methods in extension to the traditional one literary studies analysis. Important a circumstance is that, what are the PSA texts refer to freely distributed, not limited copyrighted material right and available in high quality in the email the form.

We we assume creating an independent website digital the resource that it will be in the future do not enable it critical only publication of works writer ‑ 30 volumes Pssip (1974-1983), but also others critical features materials, a also digitized manuscripts (About problems, tasks and solutions on digitization for manuscripts, see [Tchevtaev, 2, 2019, p. 27-41.). Of course, to collect all existing and emerging reference materials textual, bibliography, historical and literary comments available digitized manuscripts and link all in one semantic network – global a task that it takes time and large resources, but we believe that what is an opportunity development in this area direction it should be laid down in the the concept itself semantic analysis digital publications.

We we rely on approach to standardization e-mail address views Russian speakers literary works texts submitted by the user in the Semantic sense publishing texts L. N. Tolstoy [Orekhov, 2018, p. 381-391; http://digital.tolstoy.ru/tolstoy_search/]. The first one work stage working on a project ‑ development structures markup and its features basic implementation in the TEI language that follows the syntax language rules XML markup that makes documents TEI compliant with software features software resources creation and processing and publications xml files. Any one the TEI document consists of it consists of two parts: TEI header, which one is in compliance with the with the principles TEI allows you to encode metadata source code: description publications/manuscripts, name, first name author, language text, information about encoding, changes and etc., and text the module that contains marked text information based on the genre of the text markup the text of the story or it will be different from marking letters must contain additional metadata (recipient, date and place of writing, date, etc.). With using language TEIs can be marked up named fields entities: name, place name, organizations etc., - and also conducted semantic feature markup biographical information information, some social networks categories. such as your profession, social status, etc. This approach allows you to organizations semantic analysis search for a variety of information information for the Humanities researcher, including the one that in the usual way presentation it is being processed in the form of comments, notes and links. Like this thus, the digital publication format reflects the option interpretations texts that have been completed researchers and editors publications [Bonch-Osmolovskaya, 2016, p. 1607].

Alone from the project's tasks – in addition to digital views PSA texts, develop useful links research projects digital tools, including convenient semantic search search, statistics tools, tools visualizations etc., therefore, it should be created a special one additional semantic map layer markup, available for automatic configuration computer processing.

Preparation scientific digital editions are enough difficult task, requiring usage various digital ones tools. At the moment developed by structure markup with building on the work researchers of literary studies, conducted by clarification and bid adjustment markup with using the following parameters: computer programs methods. Provisional framework annotations text corpora a writer with using elements TEI/XML includes markup of the following entities: names and titles; dates and times year; colors; properties and States and events; natural phenomena; social status; profession; animals / plants; comments; notes. For more information we have investigated the first three volumes PSS using theme-based simulations [Korshunov, 2012] ‑ the approach that automatically allows you to identify topics in the data. Conducting research with various options settings automatic theme selections allowed select a topic, which is when different settings always turned it on includes words "human", "time", "love" and "face". We investigated usage these words in the under consideration text editor with the use of computer programs technologies (used corpus Manager Voyant Tools (https://voyant-tools.org/)) and identified, what for the word "time" is characteristic usage with the verb and articulate nouns: time to joke, time to talk; time of existence, time of absence, what does it indicate if necessary accounting for various special aspects temporalities Chekhov's books texts are not allowed only verbal ones external descriptions characteristics time (objective natural reality), but also psychological aspects subjective experiences it's a matter of time. Therefore category "seasons of the year" we have replaced to a wider audience category "time".

Pssip consists of volumes as the main one structural units to include texts are included the writer for specific period. However we rely on on the structure Eni "Chekhov" (FEB) [ENI " CHEKHOV», http://feb-web.ru/feb/chekhov/default.asp], where submitted by segmented volumes for individual cases texts. At that time same time notes and comments to individual users to the texts we use include in text markup - partially in metadata, partially in the form of comments and notes in the text itself. General comments and notes for each volume they should too be enabled in the publication, so we are preparing separate description of each psip volumes.

18 volume Psip significant the source of reference, bibliography, the editorial and critical information in particular, it contains index of names and names, on the basis of which you can organize the opportunity find relevant entities in specific texts, they joint occurrence pages edition that you will learn organization social relations writer, in the context of which to analyze linguistic and literary data. Like this this approach will allow you to create an ontology texts [see Dikonov, 2012, p. 3. (in Russian). 153] writer, in which each of us has the document has its own markup, text of each document number associated with elements various pointers, which are available in the PSC (pointer aliases, name pointer and names and we believe that what is a text page the markup will be expand by enabling it new categories ‑ concepts, events, existing links in the texts of A. P. Chekhov, and samo semantic field the entire publication will expand your network for enabling account references and editorial-critical reviews texts coming out outside the scope of the Pssip. In addition, converting to the following format TEI/XML transcriptions manuscripts the writer, their draft versions, editorial offices, copies allows you to carry out additional information reconciliation with the originals, expand the database references materials, source studies and textual data comments / notes.

In what way advantages digital editions of works Chekhov? First of all, in convenience search not only for individual cases words, but also by contexts, by units segmentation options text and their attributes semantic relationships. For example, on one of the conferences young scientist made a report about paralinguistic programs features speeches of clergymen in the stories and novels Chekhov. For listeners there was a logical result question, are peculiar whether identified features only this one a group of characters. Within the project find an answer it will be possible much faster, than manually. Second, comment it will be linked to the software text link, not in at the end of the book. The same applies to photos, illustrations, films and performances etc. And that's just it the most obvious ones features.

Semantic field Chekhov Digital edition it should become the basis for research writer's texts using Digital methods Humanities that are complement and expand features traditional ones philological studies approaches, become background the new type comments, unifying agent texts and facts, complicating it and enriching understanding the text. This the publication presents interest for researchers, teachers, students, interested parties readers.
"""

In [ ]:
summarizer = AbstractiveSummarizer()

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


Downloading:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

In [ ]:
summarizer.generate_summary(TEXT_TO_SUMMARIZE) #takes about 40 seconds

Your max_length is set to 142, but you input_length is only 78. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=39)


' Submit literary and folklore text in the form the relationship between components offered more V. Shklovsky and V. Propp . About decomposition its on "such components, of which his then you can it would be fun to collect for some reasons General rules" Digital version environment as a tool philological studies research gradually becomes an important part humanitarian issues scientific knowledge .  World Wide Web sir Tim Berners-Lee expressed an idea about the need creating a new one network-semantic (Semantic Web) Since the network hypertext links not machine readable and the search is reduced to to search for individual users words or phrases, while Semantic The network must link standing ones for texts meaning and data, making it possible analyze them automatically .  Semantic field an edition is presentation text in the form related data that Express a straight line, explicit and understandable information for a computer program processing options relationship entities . Semantic 

In [ ]:
TEXT_TO_SUMMARIZE = """
Although we normally sleep through the whole night, experts on sleeping now think that this is unnatural and may not be good for us.
In an experiment carried out two decades ago, a group of people were put into a dark room for fourteen hours a day. Towards the end of the experiment most of them developed a sleeping pattern. They slept for four hours, were then awake for a few hours and slept again for four hours. This opposed the general theory that uninterrupted sleeping for eight hours is normal.
History shows that sleeping in two phases was normal for a long time. During the time when they were awake people got up, strolled around or went smoking. Some people stayed in bed, prayed or read something. Historical prayer books contain special prayers that people said between sleeping periods.
Evidence showing the two phases of our nightly sleep started disappearing at the beginning of the Industrial Revolution. By the 20th century the idea of splitting up nightly sleep vanished from western society.
During the Middle Ages people were afraid of night and stayed indoors. It was a time when criminals, prostitutes and drunks roamed the streets. Later on people got accustomed to staying up after sunset. When street lighting became widespread people began to socialize, go to inns or pubs. Paris and Amsterdam became the first cities to introduce street lighting. By the end of the 17th century over 50 European cities were lit up at night.
Over the past two centuries people have adapted to eight hours of sleep, but waking up at least once during a night’s sleep seems to be quite normal. We wake up if we are stressed or have problems. Sometimes we panic or have a nightmare. Many doctors say this is normal and proves the theory of our two-phase sleep
pattern. Other doctors suggest that waking up between two sleep phases could have regulated stress in a natural way. Historic studies show that people may have woken up after dreaming and then thought about their dreams.
Today we spend less time thinking about our other problems, so stress, anxiety and depression has increased.
"""

In [ ]:
summarizer.generate_summary(TEXT_TO_SUMMARIZE)

Your max_length is set to 142, but you input_length is only 22. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=11)


' Experts on sleeping now think that this is unnatural and may not be good for us . Two phases of our nightly sleep started disappearing at the beginning of the Industrial Revolution . By the 20th century the idea of splitting up nightly sleep vanished from western society . History shows that sleeping in two phases was normal for a long time . During the Middle Ages people were afraid of night and stayed indoors .  Today we spend less time thinking about our other problems, so stress, anxiety and depression has increased . Stress, anxiety, depression and depression have increased in recent years, experts say . Today, we spend more time thinking of our own problems than our other issues, they say .'